# Background


This notebook is to make a baseline for slot1, aspect category detection. We use glove to generate sentence vector. 

In [ ]:
train_path = Path.cwd().parent.joinpath('data/semeval-2016/train.csv')
test_path = Path.cwd().parent.joinpath('data/semeval-2016/test.csv')